In [3]:
https://pathmind.com/kr/wiki/generative-adversarial-network-gan
    
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout,LeakyReLU,Reshape
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras import utils as np_utils
from keras.layers import Input
from keras.models import Model
from keras.optimizers import adam
from keras.layers.normalization import BatchNormalization

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer='adam')

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer='adam')

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) =tf.keras.datasets.fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch): #내 컴퓨터에 사진저장
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("C:/Users/jmlee/Desktop/Jimin/data_study/CNN_fruit/dataset/photo/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

C:\Users\jmlee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.668490, acc.: 65.62%] [G loss: 0.360684]


C:\Users\jmlee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.296934, acc.: 93.75%] [G loss: 0.407852]
2 [D loss: 0.342670, acc.: 75.00%] [G loss: 0.576764]
3 [D loss: 0.408063, acc.: 65.62%] [G loss: 0.785177]
4 [D loss: 0.461540, acc.: 71.88%] [G loss: 1.152994]
5 [D loss: 0.451367, acc.: 71.88%] [G loss: 1.857921]
6 [D loss: 0.337901, acc.: 87.50%] [G loss: 2.706073]
7 [D loss: 0.073583, acc.: 100.00%] [G loss: 3.706786]
8 [D loss: 0.040591, acc.: 100.00%] [G loss: 4.362382]
9 [D loss: 0.106503, acc.: 96.88%] [G loss: 5.485498]
10 [D loss: 0.036485, acc.: 100.00%] [G loss: 5.556046]
11 [D loss: 0.050582, acc.: 96.88%] [G loss: 5.678180]
12 [D loss: 0.020724, acc.: 100.00%] [G loss: 5.813478]
13 [D loss: 0.011520, acc.: 100.00%] [G loss: 5.811993]
14 [D loss: 0.025171, acc.: 100.00%] [G loss: 5.719093]
15 [D loss: 0.011222, acc.: 100.00%] [G loss: 6.342463]
16 [D loss: 0.142185, acc.: 96.88%] [G loss: 5.789608]
17 [D loss: 0.003188, acc.: 100.00%] [G loss: 6.129339]
18 [D loss: 0.074124, acc.: 96.88%] [G loss: 6.394797]
19 [D loss:

...

4742 [D loss: 0.026219, acc.: 100.00%] [G loss: 8.558935]
4743 [D loss: 0.005797, acc.: 100.00%] [G loss: 9.261074]
4744 [D loss: 0.152252, acc.: 93.75%] [G loss: 8.858923]
4745 [D loss: 0.016400, acc.: 100.00%] [G loss: 6.422360]
4746 [D loss: 0.010042, acc.: 100.00%] [G loss: 5.745355]
4747 [D loss: 0.120998, acc.: 96.88%] [G loss: 5.892523]
4748 [D loss: 0.008149, acc.: 100.00%] [G loss: 7.504374]
4749 [D loss: 0.073348, acc.: 96.88%] [G loss: 10.221284]
4750 [D loss: 0.144709, acc.: 93.75%] [G loss: 9.672466]
4751 [D loss: 0.306420, acc.: 96.88%] [G loss: 6.890976]
4752 [D loss: 0.066076, acc.: 96.88%] [G loss: 5.063185]
4753 [D loss: 0.253927, acc.: 90.62%] [G loss: 5.661606]
4754 [D loss: 0.072632, acc.: 96.88%] [G loss: 6.943178]
4755 [D loss: 0.006453, acc.: 100.00%] [G loss: 8.630498]
4756 [D loss: 0.030190, acc.: 100.00%] [G loss: 8.655486]
4757 [D loss: 0.020857, acc.: 100.00%] [G loss: 8.348467]
4758 [D loss: 0.089144, acc.: 96.88%] [G loss: 6.487843]
4759 [D loss: 0.183176

4885 [D loss: 0.058014, acc.: 96.88%] [G loss: 6.904860]
4886 [D loss: 0.091982, acc.: 93.75%] [G loss: 5.220581]
4887 [D loss: 0.040697, acc.: 100.00%] [G loss: 5.555587]
4888 [D loss: 0.043750, acc.: 100.00%] [G loss: 4.287996]
4889 [D loss: 0.049369, acc.: 100.00%] [G loss: 5.504028]
4890 [D loss: 0.090049, acc.: 96.88%] [G loss: 7.062683]
4891 [D loss: 0.022479, acc.: 100.00%] [G loss: 7.326795]
4892 [D loss: 0.053720, acc.: 96.88%] [G loss: 6.377154]
4893 [D loss: 0.126634, acc.: 96.88%] [G loss: 6.226695]
4894 [D loss: 0.024060, acc.: 100.00%] [G loss: 5.234039]
4895 [D loss: 0.040469, acc.: 100.00%] [G loss: 4.751490]
4896 [D loss: 0.045126, acc.: 96.88%] [G loss: 5.838166]
4897 [D loss: 0.108077, acc.: 93.75%] [G loss: 4.926171]
4898 [D loss: 0.021713, acc.: 100.00%] [G loss: 5.285295]
4899 [D loss: 0.098224, acc.: 93.75%] [G loss: 5.651061]
4900 [D loss: 0.037240, acc.: 100.00%] [G loss: 7.426299]
4901 [D loss: 0.035368, acc.: 100.00%] [G loss: 5.954724]
4902 [D loss: 0.065521

5028 [D loss: 0.037850, acc.: 96.88%] [G loss: 8.854059]
5029 [D loss: 0.019014, acc.: 100.00%] [G loss: 10.345518]
5030 [D loss: 0.034364, acc.: 96.88%] [G loss: 7.835932]
5031 [D loss: 0.285304, acc.: 93.75%] [G loss: 2.884217]
5032 [D loss: 0.405279, acc.: 78.12%] [G loss: 5.992446]
5033 [D loss: 0.008794, acc.: 100.00%] [G loss: 16.529034]
5034 [D loss: 0.442478, acc.: 90.62%] [G loss: 11.741587]
5035 [D loss: 0.167367, acc.: 96.88%] [G loss: 6.961611]
5036 [D loss: 0.607522, acc.: 78.12%] [G loss: 3.960213]
5037 [D loss: 0.208475, acc.: 93.75%] [G loss: 4.340142]
5038 [D loss: 0.032530, acc.: 100.00%] [G loss: 7.105872]
5039 [D loss: 0.072774, acc.: 96.88%] [G loss: 9.741757]
5040 [D loss: 0.111120, acc.: 93.75%] [G loss: 10.519836]
5041 [D loss: 0.208293, acc.: 96.88%] [G loss: 10.103683]
5042 [D loss: 0.089777, acc.: 93.75%] [G loss: 7.134799]
5043 [D loss: 0.022151, acc.: 100.00%] [G loss: 5.035577]
5044 [D loss: 0.037618, acc.: 100.00%] [G loss: 4.123727]
5045 [D loss: 0.23228

5172 [D loss: 0.025410, acc.: 100.00%] [G loss: 7.490312]
5173 [D loss: 0.023886, acc.: 100.00%] [G loss: 7.786787]
5174 [D loss: 0.255832, acc.: 93.75%] [G loss: 6.370121]
5175 [D loss: 0.160557, acc.: 93.75%] [G loss: 5.187202]
5176 [D loss: 0.178409, acc.: 90.62%] [G loss: 4.711407]
5177 [D loss: 0.140314, acc.: 93.75%] [G loss: 6.744423]
5178 [D loss: 0.067783, acc.: 96.88%] [G loss: 7.866619]
5179 [D loss: 0.006379, acc.: 100.00%] [G loss: 8.728666]
5180 [D loss: 0.469608, acc.: 84.38%] [G loss: 5.422927]
5181 [D loss: 0.055873, acc.: 100.00%] [G loss: 3.666717]
5182 [D loss: 0.220826, acc.: 87.50%] [G loss: 3.871881]
5183 [D loss: 0.164776, acc.: 96.88%] [G loss: 5.329867]
5184 [D loss: 0.020386, acc.: 100.00%] [G loss: 6.881969]
5185 [D loss: 0.013491, acc.: 100.00%] [G loss: 8.036705]
5186 [D loss: 0.100181, acc.: 96.88%] [G loss: 7.189241]
5187 [D loss: 0.108142, acc.: 96.88%] [G loss: 6.117471]


KeyboardInterrupt: 